===================================
 # PEAKS DETECTION #
===================================

Read the chromatogram, generate a peak table and identify the compounds using the NIST database. 

ex de file : A-F-028-817822-droite-ReCIVA.cdf
G0/G0-1-100123.cdf

docker rm -f pyms-nist-server

In [1]:
# Choix des parametres
import os
import sys
import ipywidgets as widgets
from IPython.display import display

path_to_scr_folder = os.path.join(os.path.dirname(os.path.abspath('')), 'src')
sys.path.append(path_to_scr_folder)

from identification import sample_identification

style = {'description_width': 'initial'}


def analyse(path, file, OUTPUT_PATH, mod_time, method, mode, seuil,
            hit_prob_min, ABS_THRESHOLDS, cluster, min_distance,
            sigma_ratio, num_sigma, formated_spectra, match_factor_min):
    full_path = path+file
    if not os.path.isfile(full_path):
        print(f"Erreur : Le fichier '{file}' est introuvable dans '{path}'")
        return
    if not os.access(full_path, os.R_OK):
        print(f"Erreur: Permission refusée pour accéder à '{file}' dans \
              '{path}'")
        return

    print(f"Analyzing with path={full_path}, mod_time={mod_time}, cluster={cluster}\n")
    sample_identification(path, file, OUTPUT_PATH, mod_time, method, mode,
                          seuil, hit_prob_min, ABS_THRESHOLDS, cluster,
                          min_distance, sigma_ratio, num_sigma,
                          formated_spectra, match_factor_min)


txt = widgets.HTML('<b>Choisissez vos parametres:</b>')
default_path = '/media/camille/DATA1/cdf centroid/'
path = widgets.Text(description="<b>Path</b>", value=default_path)
file = widgets.Text(description="<b>File</b>")
mod_time = widgets.FloatSlider(value=1.25, min=1, max=8, step=0.01,
                               description="<b>Modulation time:</b>",
                               style=style)
OUTPUT_PATH = widgets.Text(value=None, description="<b>new file name</b>",
                           style=style)
method = widgets.RadioButtons(
    options=['persistent_homology', 'peak_local_max', 'LoG', 'DoG', 'DoH'],
    value='persistent_homology',
    description='<b>Method</b>',
    disabled=False)
mode = widgets.RadioButtons(
    options=['tic', 'mass_per_mass', '3D'],
    value='tic',
    description='<b>Mode</b>',
    disabled=False)
seuil = widgets.IntSlider(value=5, min=0, max=10,
                            description="<b>Seuil</b>")

hit_prob_min = widgets.IntSlider(value=15, min=0, max=30,
                                   description="<b>Minimum hit probability</b> [a modifier]",
                                   style=style)

txt_abs_threshold_cluster = widgets.HTML("If mass_per_mass or 3D mode:") #TODO
# ABS_THRESHOLDS = widgets.Text(value=None, description="<b>Seuils absolus</b> [à modifier]",
#                               disabled=True, 
#                               layout=widgets.Layout(margin='0 0 0 50px'),
#                               style=style)
ABS_THRESHOLDS = widgets.Text(
    value=None,
    disabled=True,
    description="Seuils absolus(liste)",
    style=style
)

def parse_abs_thresholds(change):
    try:
        values = [float(x.strip()) for x in change['new'].split(",")]
        print("Valeurs converties :", values)
    except ValueError:
        print("Erreur : Veuillez entrer des nombres séparés par des virgules.")


ABS_THRESHOLDS.observe(parse_abs_thresholds, names='value')

cluster = widgets.Checkbox(
    value=False,
    description='<b>cluster</b>',
    disabled=True,)


def update_ABS_THRESHOLDS(change):
    if change.new == 'tic':
        ABS_THRESHOLDS.disabled = True
        cluster.disabled = True
    else:
        ABS_THRESHOLDS.disabled = False
        cluster.disabled = False


mode.observe(update_ABS_THRESHOLDS, names='value')

txt_plm = widgets.HTML("Peak local max parameter: ")
txt1 = "The minimal allowed distance separating peaks."
txt2 = "To find the maximum number of peaks, use min_distance=1."
txt_plm_description = widgets.HTML(f"({txt1} {txt2})")
min_distance = widgets.IntSlider(value=1, min=0, max=30, step=1, 
                                 description="<b>Minimal distance</b> [a modifier]",
                                 style=style)

txt_dog = widgets.HTML("DoG parameter: ")
txt3 = "The ratio between the standard deviation of Gaussian Kernels used for"\
        " computing the Difference of Gaussians."
txt_dog_description = widgets.HTML(txt3)
sigma_ratio = widgets.FloatSlider(value=1.6, min=0, max=2, step=0.01,
                                  description="<b>sigma ratio:</b>",
                                  style=style)

txt_log_doh = widgets.HTML("LoG and DoG parameter")
txt4 = "(The number of intermediate values of standard deviations to consider"\
    " between min_sigma (1) and max_sigma (30))"
txt_log_doh_description = widgets.HTML(txt4)
num_sigma = widgets.IntSlider(value=10, min=1, max=30,
                              description="<b>sigma:</b>")

formated_spectra = widgets.Checkbox(
    value=True,
    description='<b>formated spectra</b>',
    disabled=False,
)

match_factor_min = widgets.IntSlider(value=700, min=0, max=1000, step=1,
                                     description="<b>Match factor min</b> [a modifier]")

run_button = widgets.Button(description="Analyse")

output = widgets.Output()


def on_button_click(b):
    with output:
        output.clear_output()
        analyse(path.value, file.value, OUTPUT_PATH.value, mod_time.value,
                method.value, mode.value, seuil.value, hit_prob_min.value,
                ABS_THRESHOLDS.value, cluster.value, min_distance.value,
                sigma_ratio.value, num_sigma.value, formated_spectra.value,
                match_factor_min.value)


run_button.on_click(on_button_click)

vbox = widgets.VBox(layout=widgets.Layout(margin='10px 0px'))

display(txt, path, file, OUTPUT_PATH, mod_time, method, mode, seuil,
        hit_prob_min, formated_spectra, match_factor_min,
        vbox, txt_abs_threshold_cluster, ABS_THRESHOLDS, cluster, vbox,
        txt_plm, txt_plm_description, min_distance, vbox, txt_dog,
        txt_dog_description, sigma_ratio, vbox, txt_log_doh,
        txt_log_doh_description, num_sigma, vbox,
        run_button, output)



HTML(value='<b>Choisissez vos parametres:</b>')

Text(value='/media/camille/DATA1/cdf centroid/', description='<b>Path</b>')

Text(value='', description='<b>File</b>')

Text(value='', description='<b>new file name</b>', style=DescriptionStyle(description_width='initial'))

FloatSlider(value=1.25, description='<b>Modulation time:</b>', max=8.0, min=1.0, step=0.01, style=SliderStyle(…

...

HTML(value='(The number of intermediate values of standard deviations to consider between min_sigma (1) and ma…

IntSlider(value=10, description='<b>sigma:</b>', max=30, min=1)

VBox(layout=Layout(margin='10px 0px'))

Button(description='Analyse', style=ButtonStyle())

Output()